In [1]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
from pandas import Series, DataFrame

# Token

### pbmc 对于geneformer，每次的数据都需要做这个token处理

In [2]:
pbmc = sc.read_h5ad('/mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/train_pbmc.h5ad')

name_id = pd.read_csv('/mnt/nfs/wbzhang/data/celltype_annotation/gene_info_table.csv')
common_genes = list(set(name_id['gene_name']).intersection(set(pbmc.var_names)))
print(len(common_genes))
pbmc = pbmc[:, common_genes]

/mnt/nfs/wbzhang/geneformer-master/Geneformer/NEW_env/lib/python3.8/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/mnt/nfs/wbzhang/geneformer-master/Geneformer/NEW_env/lib/python3.8/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


6702


In [5]:
for i in ['IFIT1', 'IFI6', 'ISG15', 'IFIT3', 'ISG20']:
    print(i in common_genes)

True
True
True
True
True


In [7]:
ensembl_id = []
index = []
ind = 0
for gene in pbmc.var_names:
    id = name_id[name_id.gene_name == gene].ensembl_id.values[0]
    if id in ensembl_id:
        pass
    else:
        index.append(ind)
        ensembl_id.append(id)
    ind += 1

pbmc = pbmc[:, index]
pbmc.var_names = ensembl_id
pbmc.var['ensembl_id'] = pbmc.var_names

In [ ]:
pbmc = sc.read_h5ad('/mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/train_pbmc.h5ad')

name_id = pd.read_csv('/mnt/nfs/wbzhang/data/celltype_annotation/gene_info_table.csv')
common_genes = list(set(name_id['gene_name']).intersection(set(pbmc.var_names)))
print(len(common_genes))
pbmc = pbmc[:, common_genes]

for i in ['IFIT1', 'IFI6', 'ISG15', 'IFIT3', 'ISG20']:
    print(i in common_genes)

# 将gene_id转换为ensembl_id
ensembl_id = []
index = []
ind = 0
for gene in pbmc.var_names:
    id = name_id[name_id.gene_name == gene].ensembl_id.values[0]
    if id in ensembl_id:
        pass
    else:
        index.append(ind)
        ensembl_id.append(id)
    ind += 1

pbmc = pbmc[:, index]
pbmc.var_names = ensembl_id
pbmc.var['ensembl_id'] = pbmc.var_names

#必须事先保存成 .loom的格式才能去tokenize
pbmc.write_loom('/mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/tokenized_data/pbmc.loom', write_obsm_varm=True)

### Tokenize 的代码

In [13]:
from geneformer import TranscriptomeTokenizer
data_path = '/mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/tokenized_data/'
#organ_list = ['Kidney', 'Liver', 'Pancreas', 'pbmc']
loom_data_directory = data_path
output_directory = loom_data_directory
output_prefix = 'tokenized_pbmc'
tk = TranscriptomeTokenizer({'condition':'condition', "cell_type": "cell_type", 'mt_frac':'mt_frac'}, nproc=32)
tk.tokenize_data(loom_data_directory, output_directory, output_prefix)

Tokenizing /mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/tokenized_data/pbmc.loom
/mnt/nfs/wbzhang/data/perturber/experiment/data/pbmc/tokenized_data/pbmc.loom has no column attribute 'filter_pass'; tokenizing all cells.


Map (num_proc=32):   0%|          | 0/16893 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/16893 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16893 [00:00<?, ? examples/s]

In [14]:
tk

In [88]:
dixit

View of AnnData object with n_obs × n_vars = 5500 × 18531
    obs: 'cluster', 'guide', 'INTERGENIC1144056', 'INTERGENIC1216445', 'INTERGENIC216151', 'INTERGENIC393453', 'sgCREB1_2', 'sgCREB1_4', 'sgCREB1_5', 'sgE2F4_6', 'sgE2F4_7', 'sgEGR1_2', 'sgEGR1_3', 'sgEGR1_4', 'sgELF1_1', 'sgELF1_2', 'sgELF1_4', 'sgELF1_5', 'sgELK1_1', 'sgELK1_6', 'sgELK1_7', 'sgETS1_3', 'sgETS1_5', 'sgGABPA_1', 'sgGABPA_9', 'sgIRF1_2', 'sgIRF1_3', 'sgNR2C2_2', 'sgNR2C2_3', 'sgNR2C2_5', 'sgYY1_10', 'sgYY1_3', 'batch', 'n_genes', 'n_genes_by_counts', 'n_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'perturbation_name', 'perturbation_type', 'perturbation_value', 'perturbation_unit'
    var: 'index', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'ensembl_id'
    uns: 'doi', 'hvg', 'leiden', 'neighbors', 'pca', 'preprocessing_nb_link', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 

# Dixit

In [21]:
dixit = sc.read_h5ad('/mnt/nfs/wbzhang/gears/GEARS/data/Dixit_2016.h5ad')
select_list = list(map(lambda x: x in ['YY1', 'control'], list(dixit.obs['perturbation_name'])))
dixit = dixit[select_list]
dixit = dixit[list(np.arange(len(dixit))[np.array(dixit.obs['cluster'] == 'tfs_highmoi')])]

dixit.var['ensembl_id'] = list(map(lambda x:x[:15], list(dixit.var['index'])))
dixit.obs = dixit.obs.rename(columns={'total_counts':'n_counts'})

from collections import Counter
dixit.write_loom('/mnt/nfs/wbzhang/data/perturber/experiment/data/dixit/dixit.loom', write_obsm_varm=True)

/tmp/ipykernel_90828/3707139561.py:6: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  dixit.var['ensembl_id'] = list(map(lambda x:x[:15], list(dixit.var['index'])))


In [91]:
from geneformer import TranscriptomeTokenizer
data_path = '/mnt/nfs/wbzhang/data/perturber/experiment/data/dixit/'
#organ_list = ['Kidney', 'Liver', 'Pancreas', 'pbmc']
loom_data_directory = data_path
output_directory = loom_data_directory
output_prefix = 'tokenized_dixit'
tk = TranscriptomeTokenizer({'perturbation_name':'condition'}, nproc=32)
tk.tokenize_data(loom_data_directory, output_directory, output_prefix)

Tokenizing /mnt/nfs/wbzhang/data/perturber/experiment/data/dixit/dixit.loom
/mnt/nfs/wbzhang/data/perturber/experiment/data/dixit/dixit.loom has no column attribute 'filter_pass'; tokenizing all cells.


Map (num_proc=32):   0%|          | 0/5500 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/5500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5500 [00:00<?, ? examples/s]